### Preprocessing for Rignet Data

In this notebook, we process the input meshes into two mesh-graphs represented as adjacency lists. The first mesh graph contains the one-ring neighborhood as the edges of each vertex, and the second mesh graph contains the geodesic neighbord of the edges of each vertex. 

In [1]:
import sys
sys.path.insert(1, '../utils')

In [9]:
import os
import glob
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import pickle
import heapq
import open3d as o3d
import trimesh
from mesh_utils import load_and_preprocess_mesh

In [ ]:
def convert_mesh_to_graph(obj_path: str):
    """
    Reads an obj file and converts it to:
      - vertices: list of (x,y,z) coords
      - adjacency: dict mapping vertex index to list of (neighbor_index, distance)
    
    We first collect all 'v ' lines into `vertices`, and all 'f ' lines into `faces` as
    lists of integer vertex indices. Then we build the adjacency.

    return dictionary G = {
        "obj_path": obj_path,
        "vertices": vertices,
        "one_ring_distances": adjaceny dict of tuples (node, distance),
        "one_ring": edge list
    }
    """

    mesh, centroid = load_and_preprocess_mesh(obj_path,
                                              min_verts=1000,
                                              min_tris=1000,
                                              max_tris=8000)

    vertices = np.asarray(mesh.vertices)
    faces = np.asarray(mesh.faces)
    adjacency = defaultdict(list)
    
    # build adjacency from faces
    total_edge_lengths = 0.0
    num_edges = 0
    for idxs in faces:
        # assume triangular faces
        n = len(idxs)
        for j in range(n):
            u = idxs[j]
            v = idxs[(j + 1) % n]
            dist = np.linalg.norm(np.subtract(vertices[u], vertices[v]))
            total_edge_lengths += dist
            num_edges += 1
            adjacency[u].append((v, float(dist)))
            adjacency[v].append((u, float(dist)))
    
    # compute mean edge length
    mean_edge_length = (total_edge_lengths / num_edges) if num_edges else 0.0
    
    # deduplicate each adjacency list
    # Edge List (actually used by the network)

    edge_list = []
    for node, nbrs in adjacency.items():
        unique_nbrs = list(set(nbrs))
        adjacency[node] = unique_nbrs
        edge_list += [[node, nbr[0]] for nbr in unique_nbrs]

    # Add a self‑edge [i, i] for every vertex
    # to deal with max pooling edge cases
    for i in range(len(vertices)):
        edge_list.append([i, i])

    # G = (V, E)
    G = {
        "vertices": vertices,
        "num_faces": len(faces),
        "one_ring_distances": dict(adjacency),
        "one_ring": edge_list,
        "centroid": centroid
    }

    return G



### Geodesic Neighbors

In [11]:
def get_geodesic_adjacency_graph_from_mesh_graph(
    adjacency_distances: dict[int, list[tuple[int, float]]],
    geodesic_distance: float = 0.06
):
    """
    Given:
      adjacency: dict[node] -> list of (neighbor, edge_length)
      geodesic_distance: maximum path-length to consider
    
    Returns:
      edge_list
    """
    geodesic_edge_dict: dict[int, list[int]] = {}

    # For each start node, run a Dijkstra‐like expansion until distance > threshold
    for start in adjacency_distances:
        # min‐heap of (cum_distance, node)
        heap: list[tuple[float, int]] = [(0.0, start)]
        visited_dist: dict[int, float] = {start: 0.0}
        
        while heap:
            cum_dist, node = heapq.heappop(heap)
            # if this popped entry is stale (we found a shorter path before), skip
            if cum_dist > visited_dist[node]:
                continue
            # explore neighbors
            for nbr, edge_len in adjacency_distances[node]:
                new_dist = cum_dist + edge_len
                # if within threshold and either unvisited or found shorter path
                if new_dist <= geodesic_distance and (nbr not in visited_dist or new_dist < visited_dist[nbr]):
                    visited_dist[nbr] = new_dist
                    heapq.heappush(heap, (new_dist, nbr))
        
        # All visited_dist.keys() (excluding the start itself, if desired) are within the radius
        # We’ll include the start too, since geodesic radius 0 includes itself
        geodesic_edge_dict[start] = list(visited_dist.keys())

    edge_list = []
    for node, nbrs in geodesic_edge_dict.items():
        edge_list += [[node, nbr] for nbr in nbrs]

    return edge_list


### Label Lists (Joint Locations)

In [12]:
def get_joint_locations(rig_path, centroid):
    # extraxt join locations
    # disregard bone info. Only want joint locations
    joints = []
    with open(rig_path, "r") as f:
        tokens = f.readline().split()
        while(tokens[0] == "joints"):
            joints.append(list(map(float, tokens[2:])))
            tokens = f.readline().split()

    joints = np.array(joints) - centroid
    
    return joints

### Attention Masks

In [13]:
def get_attn_mask(attn_mask_path):
    with open(attn_mask_path, "r") as f:
        mask = list(map(int, f.read().splitlines()))
    return np.array(mask)

### Save Train, Test, and Dev Sets

In [14]:
data_root = "../data/ModelResource_RigNetv1_preproccessed"
obj_folder = f'{data_root}/obj'
rig_folder = f'{data_root}/rig_info'
attn_mask_folder = f'{data_root}/attn_masks'

mesh_graphs_folder = f'{data_root}/mesh_graphs'

In [ ]:
MAX_GEODESIC_DISTANCE = 0.06

for training_split in ['train', 'test', 'val']:

    training_split_file = f'{data_root}/{training_split}_final.txt'
    with open(training_split_file, "r") as file:
        mesh_indices = list(map(int, file.readlines()))

    graph_list = []
    for mesh_idx in tqdm(mesh_indices):
        obj_path = f'{obj_folder}/{mesh_idx}.obj'
        rig_path = f'{rig_folder}/{mesh_idx}.txt'
        attn_mask_path = f'{attn_mask_folder}/{mesh_idx}.txt'


        G = convert_mesh_to_graph(obj_path)

        G['geodesic'] = get_geodesic_adjacency_graph_from_mesh_graph(
            G['one_ring_distances'],
            MAX_GEODESIC_DISTANCE
        )
        del G['one_ring_distances']
        G['joints'] = get_joint_locations(rig_path, G['centroid'])
        G['attn_mask'] = get_attn_mask(attn_mask_path)

        # Reshape for convenience, and WAY faster
        G['one_ring'] = np.asarray(G['one_ring']).T
        G['geodesic'] = np.asarray(G['geodesic']).T

        G["mesh_index"] = mesh_idx
        graph_list.append(G)
    
    training_split_mesh_graph_file = f'{mesh_graphs_folder}/{training_split}.pkl'
    with open(training_split_mesh_graph_file, "wb+") as file:
        pickle.dump(graph_list, file)
        print("Saving to", training_split_mesh_graph_file)

100%|██████████| 2163/2163 [04:49<00:00,  7.47it/s]


Saving to ../data/ModelResource_RigNetv1_preproccessed/mesh_graphs/train.pkl


100%|██████████| 270/270 [00:39<00:00,  6.91it/s]


Saving to ../data/ModelResource_RigNetv1_preproccessed/mesh_graphs/test.pkl


100%|██████████| 270/270 [00:36<00:00,  7.38it/s]

Saving to ../data/ModelResource_RigNetv1_preproccessed/mesh_graphs/val.pkl


### Visualize Adjacency Lists

In [1]:
def points_to_spheres(points, color=[0, 1, 1]):
    spheres = []
    for (x, y, z) in np.asarray(points):
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
        sphere.translate((x, y, z))
        sphere.paint_uniform_color(color)
        spheres.append(sphere)
    
    return spheres

def visualize_mesh_graph(vertices: np.ndarray,
                        edge_list: np.ndarray,
                        joints_gt: np.ndarray = None, 
                        mesh: o3d.geometry.TriangleMesh = None):

    pts = vertices.astype(dtype=np.float64)
    lines = edge_list.astype(dtype=np.int32)
    
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(pts),
        lines=o3d.utility.Vector2iVector(lines)
    )

    colors = [[0.6, 0.6, 0.6] for _ in lines]
    line_set.colors = o3d.utility.Vector3dVector(colors)
    
    to_draw = []
    to_draw.append(line_set)

    if joints_gt is not None:
        to_draw.extend(points_to_spheres(joints_gt, color=[0, 1, 1]))

    if mesh is not None:
        mesh.compute_vertex_normals()
        mesh.paint_uniform_color([0.8, 0.8, 0.8])
        to_draw.append(mesh)
    
    o3d.visualization.draw_geometries(to_draw,
                                      mesh_show_back_face=True,
                                      window_name="Mesh Graph",
                                      width=800, height=600)

NameError: name 'np' is not defined

In [9]:
# Sanity Check
file_path = f'{mesh_graphs_folder}/test.pkl'
with open(file_path, "rb") as file:
    _graph_list = pickle.load(file)
file_path

'../data/ModelResource_RigNetv1_preproccessed/mesh_graphs/test.pkl'

In [10]:
type(_graph_list)

list

In [11]:
visualize_mesh_graph(np.array(_graph_list[0]['vertices']), 
                     np.array(_graph_list[0]['one_ring']),
                     np.array(_graph_list[0]['joints']))

2025-05-08 08:53:41.122 python3[51191:2955283] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-08 08:53:41.122 python3[51191:2955283] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [ ]:
visualize_mesh_graph(np.array(_graph_list[0]['vertices']), 
                     np.array(_graph_list[0]['geodesic']),
                     np.array(_graph_list[0]['joints']))

In [12]:
_graph_list[0].keys()

dict_keys(['obj_path', 'vertices', 'one_ring', 'centroid', 'geodesic', 'joints', 'attn_mask'])